#Import libraries
 



In [1]:
import numpy as np
import nibabel as nib
import nibabel.processing as nibp
import os

# Make masks

Make gray matter mask

In [2]:
# load functional data
filepath_func = 'sub-01_task-sherlockPart1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'
func = nib.load(filepath_func)
print(func.affine)

FileNotFoundError: ignored

In [ ]:
# load gray matter
filepath_gm = './sub-01_space-MNI152NLin2009cAsym_label-GM_probseg.nii.gz'
gm = nib.load(filepath_gm)
print(gm.affine)

In [ ]:
# resize gray matter
func0 = func.slicer[:,:,:,0]
gm_funcSize=nibp.resample_from_to(gm, func0)
print(gm_funcSize.affine)

In [ ]:
# discretize gray matter
gm_values = gm_funcSize.get_fdata()
gm_mask = (gm_values>0.1)

Make mask for regions of no interest ("confounds mask")

In [ ]:
# load white matter and csf
filepath_wm = 'sub-01_space-MNI152NLin2009cAsym_label-WM_probseg.nii.gz'
wm = nib.load(filepath_wm)
print(wm.affine)
filepath_csf = 'sub-01_space-MNI152NLin2009cAsym_label-CSF_probseg.nii.gz'
csf = nib.load(filepath_csf)
print(csf.affine)

In [ ]:
# resize white matter and csf
wm_funcSize=nibp.resample_from_to(wm, func0)
print(wm_funcSize.affine)
csf_funcSize=nibp.resample_from_to(csf, func0)
print(csf_funcSize.affine)

In [ ]:
# discretize white matter and csf and make confounds mask
wm_values = wm_funcSize.get_fdata()
csf_values = csf_funcSize.get_fdata()
confounds_values = (wm_values+csf_values)/2
confounds_mask = (confounds_values>0.1)

# Extract functional data

In [ ]:
# Extract functional data from masks
func_values = func.get_fdata()
func_reshaped = np.reshape(func_values,[func.shape[0]*func.shape[1]*func.shape[2],func.shape[3]])
gm_reshaped = np.reshape(gm_mask,-1)
confounds_mask_reshaped = np.reshape(confounds_mask,-1)
func_gm = func_reshaped[gm_reshaped,:] # these are the functional data in gray matter
func_confounds = func_reshaped[confounds_mask_reshaped,:] # these are the functional data in the regions of no interest
print(func_gm.shape)
print(func_confounds.shape)

# Calculate principal components of confounds

In [ ]:
# PCA likes the time dimension as first. Let's transpose our data.
func_gm = np.transpose(func_gm)
func_confounds = np.transpose(func_confounds)
# Fit PCA and extract PC timecourses
pca = PCA(n_components = 5)
pca.fit(func_confounds)
confounds_pc = pca.fit_transform(func_confounds)
confounds_pc.shape

# Regress confounds out of the data

In [ ]:
# linear regression on each voxel: PCs -> voxel pattern
linear = linear_model.LinearRegression()
linear.fit(confounds_pc, func_gm)

# predict the activity of each voxel for this run 
predict = linear.predict(confounds_pc)
func_denoised = func_gm - predict # t x v
func_denoised = np.transpose(func_denoised) # v x t
func_denoised.shape

# Save the denoised data

In [ ]:
func_final = np.zeros([func.shape[0]*func.shape[1]*func.shape[2],func.shape[3]])
func_final[gm_reshaped,:] = func_denoised
func_final = np.reshape(func_final,[func.shape[0],func.shape[1],func.shape[2],func.shape[3]])

In [ ]:
new_header = func.header.copy()
denoised_img = nib.nifti1.Nifti1Image(func_final, None, header=new_header)
nib.save(denoised_img, os.path.join('./','denoised.nii.gz'))
nib.save(denoised_img, os.path.join('./','denoised.nii.gz'))
